Load Env

In [13]:
from dotenv import load_dotenv

load_dotenv()

True

Get Embeddings

In [14]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=256)

Get Retriever

In [15]:
from langchain_chroma import Chroma
import os 
vector_store = Chroma(
    collection_name="adaptive_rag_test_3",
    embedding_function=embedding,
    chroma_cloud_api_key=os.getenv("CHROMA_API_KEY"),
    tenant=os.getenv("CHROMA_TENANT"),
    database=os.getenv("CHROMA_DATABASE")
)

top_k = 3
retriever = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": top_k},
    )

Define LLM

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-5-nano", stream_usage=True, temperature=1, reasoning_effort="minimal"
)

Create Contextualize Chain

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    Runnable,
    RunnablePassthrough,
    RunnableParallel,
    chain,
)
from operator import itemgetter

contextualize_instructions = """Convert the latest user question into a standalone question given the chat history. Don't answer the question, return the question and nothing else (no descriptive text)."""
contextualize_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", contextualize_instructions),
          ("placeholder", "{chat_history}"),
          ("human", "{question}"),
      ]
  )
contextualize_question = contextualize_prompt | llm | StrOutputParser()

@chain
def contextualize_if_needed(input_: dict) -> Runnable:
    if input_.get("chat_history"):
        return contextualize_question
    else:
        return RunnablePassthrough() | itemgetter("question")

Create QA Chain

In [18]:
instruction = "Answer the questions using the given context only."

qa_instructions = instruction + """\n\n{context}."""
qa_prompt = ChatPromptTemplate.from_messages(
  [("system", qa_instructions), ("human", "{question}")]
)

def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)

formatted_prompt = {
    "question": itemgetter("question") | RunnablePassthrough(),
    "context": lambda x: format_docs(x["context"]),
} | RunnableParallel(prompt=qa_prompt, question=itemgetter("question"))

qa_chain = formatted_prompt | RunnableParallel(
    llm_result=itemgetter("prompt") | llm | StrOutputParser(),
    question=itemgetter("question"),
    )

Retriever Chain

In [19]:
retrieve_docs_chain = itemgetter("question") | retriever

Create Final Chain (Contextualize -> Retrieval -> Corrective)

In [20]:
final_chain = (
    RunnablePassthrough.assign(question=contextualize_if_needed)
    .assign(context=retrieve_docs_chain)
    .assign(answer=qa_chain)
)

Invoke Chain

In [21]:
input = "Howm many types of fixed deposit does Brillar Bank offer?"
result = final_chain.invoke(
        {"question": input, "chat_history": []},
    )

answer = result["answer"]["llm_result"]
source_documents = result["context"]

print("# Answer:\n", answer)
print("\n------------\n")
for doc in source_documents:
    print(doc)
    print("\n")

# Answer:
 Brillar Bank offers four types of fixed deposit products:
- Fixed Deposit (FD)
- E-Fixed Deposit (E-FD)
- Flexi Fixed Deposit (Flexi FD)

------------

page_content='Fixed Deposit | Brillar Bank

























top of page This website was built on Wix. Create yours today.Get StartedHomeDepositsMoreUse tab to navigate through the menu items.Available for Registered Users       Learn More 




Fixed Deposit @Brillar BankFeatures and BenefitsBrillar Bank Fixed Deposit account is a good start for your long-term savings plan.Get StartedLearn More 




Features and BenefitsLong TermA choice of terms from 1 - 60 months.Flexibility of early fixed deposit partial withdrawal without losing interest on remaining balanceInterest PayoutReceive interest at maturity.Convenient WithdrawalYou can withdraw from any branch nationwide.Partial withdrawal is in multiples of RM1,000. Outstanding balances will be shown on the monthly e-statement.


Effective 1 January 2019, no interest sh

In [22]:
input = "What are the interest rates for senior flexi fixed deposit in Brillar Bank?"
result = final_chain.invoke(
        {"question": input, "chat_history": []},
    )

answer = result["answer"]["llm_result"]
source_documents = result["context"]

print("# Answer:\n", answer)
print("\n------------\n")
for doc in source_documents:
    print(doc)
    print("\n")

# Answer:
 Brillar Bank – Senior Savers Flexi Fixed Deposit interest rates by tenor:
- 1 month: 2.15%
- 2–3 months: 2.25%
- 4–5 months: 2.30%
- 6 months: 2.30%
- 7–11 months: 2.35%
- 12–60 months: 2.50%

------------

page_content='Senior Savers Flexi Fixed Deposit | Brillar Bank

























top of page This website was built on Wix. Create yours today.Get StartedHomeDepositsMoreUse tab to navigate through the menu items.Available for Registered Users       Learn More 




Seniors Savers Flexi Fixed Deposit @Brillar BankA flexible fixed deposit account for golden agerSenior Savers Flexi Fixed Deposit account is designed for individuals aged 50 years and above. It gives attractive interest and allows partial withdrawal anytime without losing your FD interest on account balance.Get StartedLearn More 




Features and BenefitsEarn attractive interest, credited at maturity into any Brillar Deposit Savings or Current AccountMonthly consolidated statementFor easy account managem

In [24]:
input = "Show me the interest rates for e-fixed deposit, flexi fixed deposit, general fixed deposit, and senior flexi fixed deposit at Brillar Bank?"
result = final_chain.invoke(
        {"question": input, "chat_history": []},
    )

answer = result["answer"]["llm_result"]
source_documents = result["context"]

print("# Answer:\n", answer)
print("\n------------\n")


# Answer:
 Here are the Brillar Bank interest rates shown in the provided information:

- E-Fixed Deposit (eFD)
  - 1 month: 1.85%
  - 2 months: 2.25%
  - 3 months: 2.25%
  - 4 months: 2.30%
  - 5 months: 2.30%
  - 6 months: 2.30%
  - 7 months: 2.35%
  - 8 months: 2.35%
  - 9 months: 2.35%
  - 10 months: 2.35%
  - 11 months: 2.35%
  - 12 months: 2.35%
  - 13 – 60 months: 2.35%

- Senior Savers Flexi Fixed Deposit
  - 1 month: 2.15%
  - 2 – 3 months: 2.25%
  - 4 – 5 months: 2.30%
  - 6 months: 2.30%
  - 7 – 11 months: 2.35%
  - 12 – 60 months: 2.50%

- Fixed Deposit (general, Brillar Bank Fixed Deposit)
  - 1 month: 2.15%
  - 2 months: 2.25%
  - 3 months: 2.25%
  - 4 months: 2.30%
  - 5 months: 2.30%
  - 6 months: 2.30%
  - 7 months: 2.35%
  - 8 months: 2.35%
  - 9 months: 2.35%
  - 10 months: 2.35%
  - 11 months: 2.35%
  - 12 months: 2.35%
  - 13 – 60 months: 2.35%

- General Fixed Deposit appears to refer to the standard Fixed Deposit (non-senior) rates listed above as “Fixed Deposit 